In [1]:
import pandas as pd
import feather
import matplotlib.pyplot as plt
from datetime import date
import numpy as np


# Runtime Parameters

In [2]:
nuclear_shutdown = True
nuclear_reduction = 0 #as a percent of initial value
coal_shutdown = True
coal_reduction = 0
add_generators = True
modify_regional_load = False
region_load_to_modify = 'NY_Z_GI'

# Generation

## Load original inputs (with a lot of missing capacities) and eGrid plant-level database

In [3]:
# Load input files from Alan
orig_inp = pd.read_csv('./good_model_inputs/inputs_generators_v2.csv')

# Load to match missing capacities with eGRID 
egrid = feather.read_dataframe('./good_model_inputs/egrid.feather')

# Load to use raw fuelcost
#epaparsed = pd.read_excel('/home/gchossie/epa_ace/good_model_inputs/epa2015ParsedFile.xlsx')

# Load to use EIA 2016 fuel costs
eiaprice = pd.read_excel('./good_model_inputs/eia_price_data.xlsx',\
                        skiprows=4, header=0)
#Times for which we will modify our load
#load_times = pd.read_csv('./good_model_inputs/times_NY_Z_GI.csv')

# Setup output file
out = orig_inp.copy()

In [4]:
# Total capacity by type
orig_inp.groupby('FuelType')['Capacity'].sum()

FuelType
Biomass         4821.0
Coal          331112.0
Fwaste           412.0
Geothermal      2439.0
Hydro          28839.6
LF Gas          1852.0
MSW             2239.0
NaturalGas    435126.0
Non-Fossil      1589.0
Nuclear       102493.0
Oil            31285.0
Pet. Coke       1101.0
Pumps          22349.0
Solar          10842.0
Waste Coal       893.0
Wind           68058.0
Name: Capacity, dtype: float64

In [5]:
# Median fuel cost by type
orig_inp.groupby('FuelType')['FuelCostTotal'].median()

FuelType
Biomass        45.450000
Coal           25.725000
Fwaste          0.010000
Geothermal      0.010000
Hydro           0.010000
LF Gas          0.010000
MSW             0.010000
NaturalGas     30.569185
Non-Fossil      0.010000
Nuclear         8.400000
Oil           211.500000
Pet. Coke      41.868750
Pumps           0.010000
Solar           0.010000
Waste Coal     30.712500
Wind            0.010000
Name: FuelCostTotal, dtype: float64

## Limit max hydro capacity by macro region

In [6]:
regions = {'NW': ['Washington', 'Oregon', 'Idaho', 'Montana', 'Wyoming'],
          'SW': ['California', 'Nevada', 'Utah', 'Arizona', 'New Mexico', 'Colorado'],
          'MW': ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri', 'Wisconsin', 'Illinois', 'Indiana', 'Kentucky', 'Michigan', 'Ohio'],
          'SE': ['Oklahoma', 'Texas', 'Arkansas', 'Louisiana', 'Tennessee', 'Alabama', 'Mississippi', 'Georgia', 'Florida', 'North Carolina', 'South Carolina', 'Virginia'],
          'NE': ['District of Columbia', 'West Virginia', 'Maryland', 'Delaware', 'New Jersey', 'Pennsylvania', 'New York', 'Rhode Island', 'Massachusetts', 'Connecticut', 'Vermont', 'New Hampshire', 'Maine']
          }
# max factors are the upper ranges of the graph https://www.energy.gov/sites/prod/files/2018/04/f51/Hydropower%20Market%20Report.pdf p.70 (74 of 108)
# except for the NW which underpredicts Hydro, so we allow it to max out to 100% if needed
def maxhydrocapa(state):
    if state in regions['NW']:
        return 1 # 0.68 is the true highest value
    elif state in regions['SW']:
        return 0.56
    elif state in regions['MW']:
        return 0.68
    elif state in regions['SE']:
        return 0.49
    elif state in regions['NE']:
        return 0.73
    else:
        raise ValueError('Invalid state')

In [7]:
# Select hydro power plant and replace their max capacity
idx = (out.loc[out['FuelType'] == 'Hydro',:]).index
for i in idx:
    out.loc[i, 'Capacity'] *= maxhydrocapa(out.loc[i, 'StateName']) 
    
idx = (out.loc[out['FuelType'] == 'Pumps',:]).index
for i in idx:
    out.loc[i, 'Capacity'] *= maxhydrocapa(out.loc[i, 'StateName']) 

## Fix the labeling of coal, biomass, nat gas, and oil

In [8]:
fuel_abs = ['AB', 'BG', 'BLQ', 'DG', 'LFG', 'ME', 'MSB', 'OBL', 'OBS', 'PP', 'SLW', 'WDL', 'WDS',\
           'ANT', 'BIT', 'LIG', 'SUB', 'SC', 'RC', 'WC', 'SGC','NG', 'PG', 'BU',\
           'DFO', 'JF', 'KER', 'OO', 'OTL', 'PC', 'RG', 'RFO', 'WO']
fueltype_abbrev = {'Biomass': ['AB', 'BG', 'BLQ', 'DG', 'LFG', 'ME', 'MSB', 'OBL', 'OBS', 'PP', 'SLW', 'WDL', 'WDS'],\
           'Coal': ['ANT', 'BIT', 'LIG', 'SUB', 'SC', 'RC', 'WC', 'SGC'],\
           'NaturalGas': ['NG', 'PG', 'BU'],\
           'Oil': ['DFO', 'JF', 'KER', 'OO', 'OTL', 'PC', 'RG', 'RFO', 'WO']}
def find_key(value):
    return next((k for k, v in fueltype_abbrev.items() if value in v), None)

In [9]:
# Find discrepancies between FuelType and FUELU1 columns
for idx in out.index:
    fu = out.loc[idx, 'FUELU1']
    cat = out.loc[idx, 'FuelType']
    egrid_cat = find_key(fu)
    if fu in fuel_abs and cat != egrid_cat:
        cat = egrid_cat

## Fix regions names

In [10]:
out.loc[out['RegionName'] == 'NY_Z_A&B', 'RegionName'] = 'NY_Z_AB'
out.loc[out['RegionName'] == 'NY_Z_C&E', 'RegionName'] = 'NY_Z_CE'
out.loc[out['RegionName'] == 'NY_Z_G-I', 'RegionName'] = 'NY_Z_GI'

## Get fuel cost from EIA data instead of raw input from Alan

In [11]:
us_state_abbrev={'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA',\
                'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia':\
                'DC','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho':\
                'ID','Illinois': 'IL','Indiana': 'IN','Iowa':'IA','Kansas': 'KS','Kentucky': 'KY',\
                'Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA',\
                'Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO',\
                'Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH',\
                 'New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC',\
                 'North Dakota': 'ND','Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK',\
                 'Oregon': 'OR','Palau': 'PW','Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI',\
                 'South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT',\
                 'Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA','Washington': 'WA','West Virginia':\
                 'WV','Wisconsin': 'WI','Wyoming': 'WY'}

In [12]:
eiaprice.loc[eiaprice['FUEL_COST'] == '.', 'FUEL_COST'] = None
eiaprice.loc[eiaprice['FUEL_COST'] >= 1e5, 'FUEL_COST'] = eiaprice.loc[eiaprice['FUEL_COST'] <= 1e4, 'FUEL_COST'].max()

In [13]:
for gen in out.index:
    if out.loc[gen, 'SUBRGN'] not in ['RFCW', 'SRMV', 'SRMW', 'AZNM']:
        if out.loc[gen,'FuelType'] == 'Coal' or out.loc[gen,'FuelType'] == 'NaturalGas':
            # determine whether EIA has fuel data
            idx = (eiaprice['Plant Id'] == out.loc[gen,'ORISCode']).values *\
                (eiaprice['ENERGY_SOURCE'] == out.loc[gen,'FUELU1']).values
            # if this is not empty
            if pd.notnull(eiaprice.loc[idx, 'FUEL_COST'].mean()):
                out.loc[gen,'FuelCostTotal'] = eiaprice.loc[idx, 'FUEL_COST'].mean()*1e-2*\
                                out.loc[gen,'HeatRate']*1e-3
            else:# get state-level value
                idx_st = (eiaprice['Plant State'] == us_state_abbrev[out.loc[gen,'StateName']]).values*\
                    (eiaprice['ENERGY_SOURCE'] == out.loc[gen,'FUELU1']).values*\
                    (eiaprice['FUEL_COST'] > 0).values
                if pd.notnull(eiaprice.loc[idx_st, 'FUEL_COST'].mean()):
                    out.loc[gen,'FuelCostTotal'] = eiaprice.loc[idx_st, 'FUEL_COST'].mean()*1e-2*\
                                out.loc[gen,'HeatRate']*1e-3

In [14]:
# look for missing values
out[pd.isnull(out['FuelCostTotal'])]

,RegionName,FuelType,FuelCostTotal,PLCO2RTA,ORISCode,DIVISIONCE,UNITID,Capacity,OBJECTID,UniqueID,...,PLNOXRTA,PLSO2RTA,PLN2ORTA,PLCH4RTA,PRMVR,FUELU1,pm.mmbtu,pm.mwh,generationCO2,g


In [15]:
out.groupby('FuelType')['FuelCostTotal'].median()

FuelType
Biomass        45.450000
Coal           25.620000
Fwaste          0.010000
Geothermal      0.010000
Hydro           0.010000
LF Gas          0.010000
MSW             0.010000
NaturalGas     34.467779
Non-Fossil      0.010000
Nuclear         8.400000
Oil           211.500000
Pet. Coke      41.868750
Pumps           0.010000
Solar           0.010000
Waste Coal     30.712500
Wind            0.010000
Name: FuelCostTotal, dtype: float64

## Turn off Nuclear Power by deleting the Nuclear plants from the dataset


In [16]:
#create a variable to reduce our load later 
#(we will reduce based on the nuclear generation that would have been available but will now be cut)
nuclear_load_reduction = out.loc[(out['FuelType'] == 'Nuclear') & (out['RegionName'] == region_load_to_modify),'Capacity'].sum()

In [17]:
#check our initial nuclear power generation
out.loc[out['FuelType'] == 'Nuclear', 'Capacity']

496      1205.0
497      1195.0
498      1280.0
499      1280.0
1036      981.0
          ...  
12702    1314.0
12703    1312.0
14483    1132.0
16417    1122.0
16418    1118.0
Name: Capacity, Length: 102, dtype: float64

In [18]:
#shutdown nuclear power if we have set it to true above
if nuclear_shutdown == True:
    out.loc[out['FuelType'] == 'Nuclear', 'Capacity'] *= nuclear_reduction
    if nuclear_reduction == 0.:
        out= out.loc[out['FuelType'] != 'Nuclear']

In [19]:
#check we no longer have nuclear generation
out.loc[out['FuelType'] == 'Nuclear', 'Capacity']

Series([], Name: Capacity, dtype: float64)

## Turn off Coal Power if chosen

In [20]:
#shutdown nuclear power if we have set it to true above
if coal_shutdown == True:
    out.loc[out['FuelType'] == 'Coal', 'Capacity'] *= coal_reduction
    if nuclear_reduction == 0.:
        out= out.loc[out['FuelType'] != 'Coal']

In [21]:
#check we no longer have nuclear generation
out.loc[out['FuelType'] == 'Coal', 'Capacity']

Series([], Name: Capacity, dtype: float64)

## Reduce generation cost for RC (subsidies)

In [22]:
out.loc[out['FUELU1'] == 'RC', 'FuelCostTotal'] -= 3

## Get EI from eGrid instead of Alan
#### Convert from lb to kg

In [23]:
# Fix EI (take back from eGrid)
# For each unique ORISCode in Alan's dataset,
#    Get PLNOXRTA, etc from the total power plant from eGRid
not_found=0
for oris in set(out['ORISCode']):
    # Exclude the regions where the original EI work well
    #if out.loc[out['ORISCode'] == oris, 'SUBRGN'].all() not in ['NWPP', 'RFCE', 'RMPA']:
    for sp in ['NOX', 'SO2', 'CH4', 'CO2', 'N2O']:
        if egrid.loc[egrid['ORISPL'] == oris, f'PL{sp}RTA'].values.size:#ie if array not empty
            plrta = egrid.loc[egrid['ORISPL'] == oris, f'PL{sp}RTA'].values[0] * 0.45359 # kg/lb
            out.loc[out['ORISCode'] == oris, f'PL{sp}RTA'] = plrta
        else:
            not_found+=1
#Alan's original data is in kg/MWh, so we convert eGRID data to kg as well.

## Add generators
-in all regions at a cost such that it is only activated when absolutely necessary. Set the emission factors to zero. We will record when it is used and assume partial blackouts when used. This is just to close the budget and allow for optimization

-co-locate with the most expensive EGU in that region

-set the capacity at 20000 to ensure it fills the gap

-set the price at 1000 to ensure that it is the highest price and only turned on in these scenarios


In [24]:
if add_generators == True:
    #get list of unique region names
    reg = np.unique(out['RegionName'])
    generator_oris_number = np.arange(100000,100000+len(reg))
    #create a dictionary to reassign the mixed ORISCodes for our generators (99999region_name) to a number (100000 and above)
    generator_oris_dict = {f'99999{r}':generator_oris_number[idx] for idx,r in enumerate(reg)}    #create data frame for our generators
    generators = pd.DataFrame(index = (generator_oris_dict[f'99999{r}'] for r in reg), columns = out.columns.values)
    #fill the dataframe by region
    for idx,r in enumerate(reg):
        generators.loc[generator_oris_dict[f'99999{r}']] = out.loc[(out['RegionName'] == r) & (out['FuelCostTotal'] == (out.loc[(out['RegionName'] == r),'FuelCostTotal'].max()))].values[0]
        generators.loc[generator_oris_dict[f'99999{r}'],'ORISCode'] = generator_oris_dict[f'99999{r}']
        generators.loc[generator_oris_dict[f'99999{r}'],'OBJECTID'] = f'99999_{r}'
        generators.loc[generator_oris_dict[f'99999{r}'],'UniqueID'] = f'99999_{r}'
        generators.loc[generator_oris_dict[f'99999{r}'],'RegionName'] = r
    #add in fuel cost, capcity, and emissions factors (same for all generators)
    generators['FuelCostTotal'] = 1000
    generators['Capacity'] = 20000 #for nonuc only; 40000 for nonuc and no coal
    generators['PLCO2RTA'] = 0
    generators['PLNOXRTA'] = 0
    generators['PLN2ORTA'] = 0
    generators['PLSO2RTA'] = 0
    generators['PLCH4RTA'] = 0
    generators['FuelType'] = 'generator'
    generators['HeatRate'] = 0
    generators['pm.mmbtu'] = 0
    generators['pm.mwh'] = 0
    generators['generationCO2'] = 0
    generators['g'] = 0
    #append this to our generation out dataframe
    out = out.append(generators)
    out

## Save output

In [25]:
# Save 
out.to_csv(f'./good_model_inputs/inputs_gen_no-nuclear_no_coal.csv')

In [36]:
# Check for nans
out.loc[pd.isnull(out['Capacity']),:]

,RegionName,FuelType,FuelCostTotal,PLCO2RTA,ORISCode,DIVISIONCE,UNITID,Capacity,OBJECTID,UniqueID,...,PLNOXRTA,PLSO2RTA,PLN2ORTA,PLCH4RTA,PRMVR,FUELU1,pm.mmbtu,pm.mwh,generationCO2,g


# Transmission

## Load original inputs

In [31]:
trans = pd.read_csv('../good_model_inputs/inputs_transmission.csv')

## Modify inputs

In [32]:
# Separate ERCOT (Texas: ERC_FRNT, ERC_GWAY, ERC_REST, ERC_WEST) from the other regions
lst = ['ERC_FRNT', 'ERC_GWAY', 'ERC_REST', 'ERC_WEST']
idx = (trans['r1'].isin(lst) & ~trans['r2'].isin(lst) |\
        trans['r2'].isin(lst) & ~trans['r1'].isin(lst))&\
        (trans['transCap'] > 1).values
trans.loc[idx,'transCap'] = 1
trans.loc[idx,'transCost'] = 0.01

In [33]:
# Isolate WECC (West of Rockies) from the rest
lst = ['WECC_AZ','WECC_CO','WECC_ID','WECC_IID','WECC_MT', 'WECC_NM','WECC_NNV','WECC_PNW',\
       'WECC_SCE','WECC_SF','WECC_SNV','WECC_UT','WECC_WY','WEC_CALN','WEC_LADW','WEC_SDGE']
idx = (trans['r1'].isin(lst) & ~trans['r2'].isin(lst) |\
        trans['r2'].isin(lst) & ~trans['r1'].isin(lst))&\
        (trans['transCap'] > 1).values
trans.loc[idx,:]
trans.loc[idx,'transCap'] = 1
trans.loc[idx,'transCost'] = 0.01

In [35]:
# Add incoming capacity to NY_Z_J (3 active lines, add 700MW to each)
idx = (trans['r2'] == 'NY_Z_J').values * (trans['transCap'] > 1).values
trans.loc[idx, 'transCap'] += 700

## Save output

In [40]:
trans.to_csv(f'./good_model_inputs/inputs_trans_no-nuclear_{tdy}.csv')

# Load

## Load original inputs

In [22]:
#import our data
load = pd.read_csv('./good_model_inputs/inputs_load.csv')
#increase the demand load by 15% in ERC_WEST and ERC_REST
idx = (load['r'] == 'ERC_REST').values + (load['r'] == 'ERC_WEST').values
load.loc[idx, 'demandLoad'] *= 1.15


## Modify regional load (if set to true), usually not needed if we modify the generators

In [107]:
#modify the load at the particular times that generation < load (we will assume partial blackouts in these regions at these times)
if modify_regional_load == True:
    region = region_to_modify
    times = load_times['0'].values
    date_range = np.arange(5000,6002)
    print('Reducing load in regions where load exceeds generation by 170 MwH by ' + str(nuclear_load_reduction))
    print('Mean of load before modification '+ str(load.loc[(load['r'] == region),'demandLoad'].mean()))
    #reduce our loads by the amount that nucler would have supplied in all regions where it is possible; 
    #for those regions where the load stays above 546 MwH, we reduce it to 546 or else the model will not optimize
    for t in times:
        if ((load.loc[(load['r'] == region_to_modify) & (load['t'] == t),'demandLoad']) - nuclear_load_reduction).values <= 546: #alter anything below 546 load 
            load.loc[(load['r'] == region_to_modify) & (load['t'] == t),'demandLoad'] -= nuclear_load_reduction
        elif (load.loc[(load['r'] == region_to_modify) & (load['t'] == t),'demandLoad'] - nuclear_load_reduction).values > 546: # we have to assume anything above 546
            load.loc[(load['r'] == region_to_modify) & (load['t'] == t),'demandLoad'] = 546
    print('Mean of load after modification '+ str(load.loc[(load['r'] == region),'demandLoad'].mean()))
    print(load.loc[(load['r'] == region) & load['t'].isin(times)])

Reducing load in regions where load exceeds generation by 170 MwH by 2061.0
Mean of load before modification 1849.0366480191803
Mean of load after modification 1839.855462952392
              r     t  demandLoad
180186  NY_Z_GI  5007       350.0
180187  NY_Z_GI  5008       363.0
180188  NY_Z_GI  5009       343.0
180211  NY_Z_GI  5032       349.0
180212  NY_Z_GI  5033       349.0
180235  NY_Z_GI  5056       339.0
180236  NY_Z_GI  5057       353.0
180255  NY_Z_GI  5076       415.0
180256  NY_Z_GI  5077       489.0
180257  NY_Z_GI  5078       546.0
180258  NY_Z_GI  5079       546.0
180259  NY_Z_GI  5080       546.0
180260  NY_Z_GI  5081       509.0
180261  NY_Z_GI  5082       409.0
180280  NY_Z_GI  5101       360.0
180281  NY_Z_GI  5102       405.0
180282  NY_Z_GI  5103       437.0
180283  NY_Z_GI  5104       459.0
180284  NY_Z_GI  5105       457.0
180285  NY_Z_GI  5106       401.0
180353  NY_Z_GI  5174       357.0
180354  NY_Z_GI  5175       366.0
180355  NY_Z_GI  5176       380.0
180356

In [61]:
load.to_csv(f'./good_model_inputs/inputs_load_no-nuclear_{tdy}_reduced-100percent-nuclear_modified-max0_NYZJ-0.csv')